In [16]:
# This code is used to generate the maps

import os
import folium
import pandas as pd
import numpy as np
import json
from folium.plugins import MarkerCluster
from branca.utilities import split_six
from branca.colormap import linear
import geopandas as gpd

LA_COORDINATES = (34.009, -118.2424)
crimedata = pd.read_csv('Data/ViolentCrime2017.csv')
#len(crimedata)=230420
print("The version of Folium used is:")
print(folium.__version__)
print(f"The length of crimedata dataframe is: {len(crimedata)}")
crimedata.head()

The version of Folium used is:
0.6.0
The length of crimedata dataframe is: 33139


,Unnamed: 0,area_id,area_name,crm_cd,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,crm_cd_desc,cross_street,...,vict_age,vict_descent,vict_sex,weapon_desc,weapon_used_cd,Month,Year,Latitude,Longitude,Zip Code
0,3903,12,77th Street,110,110,998.0,NaN,NaN,CRIMINAL HOMICIDE,NaN,...,25.0,B,M,UNKNOWN FIREARM,106.0,9,2017,33.9638,-118.2575,90001
1,4000,12,77th Street,110,110,NaN,NaN,NaN,CRIMINAL HOMICIDE,NaN,...,26.0,H,M,HAND GUN,102.0,7,2017,33.9620,-118.2575,90001
2,4061,12,77th Street,110,110,NaN,NaN,NaN,CRIMINAL HOMICIDE,NaN,...,21.0,B,M,SEMI-AUTOMATIC PISTOL,109.0,8,2017,33.9620,-118.2575,90001
3,4132,12,77th Street,230,230,NaN,NaN,NaN,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",WADSWORTH AV,...,NaN,W,M,REVOLVER,101.0,10,2017,33.9638,-118.2586,90001
4,4276,13,Newton,624,624,NaN,NaN,NaN,BATTERY - SIMPLE ASSAULT,NaN,...,67.0,H,M,UNKNOWN WEAPON/OTHER WEAPON,500.0,9,2017,33.9857,-118.2628,90001


In [17]:
#Describe the types of violent crimes
crimedata[("crm_cd_desc")].value_counts()

BATTERY - SIMPLE ASSAULT                          18756
ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT    10801
BATTERY WITH SEXUAL CONTACT                        1255
RAPE, FORCIBLE                                     1164
BATTERY POLICE (SIMPLE)                             540
CRIMINAL HOMICIDE                                   278
ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER        188
RAPE, ATTEMPTED                                     106
BATTERY ON A FIREFIGHTER                             45
LYNCHING                                              4
LYNCHING - ATTEMPTED                                  2
Name: crm_cd_desc, dtype: int64

In [18]:
#Calculate the crime counts by zipcodes

crimerate2017= crimedata.groupby("Zip Code").count().reset_index()
crimerate2017 = crimerate2017[["Zip Code", "crm_cd"]]
crimerate2017 = crimerate2017.rename(columns={"crm_cd": "2017 Crime count"})

crimerate2017.to_csv("Data/crimerate2017.csv")
#print(f"The number of observations in this dataframe is: {len(crimerate2017)}")
#crimerate2017.head()
unique_zips = crimerate2017[("Zip Code")].value_counts().count()
#The number of zipcodes
print(f"The number of zipcodes: {unique_zips}")

The number of zipcodes: 135


In [19]:
crimerate2017.head()

,Zip Code,2017 Crime count
0,90001,51
1,90002,524
2,90003,1456
3,90004,267
4,90005,260


In [20]:
#Merge it with the Census Data to be able to calculate the violent crime rates, population density, etc. 
#Merge how = inner
#Left dataframe = crimerate2017
#Right dataframe = acs2016_5yr_B03002_Race.csv
census = pd.read_csv("Data/acs2016_5yr_B03002_Race.csv")
len(census)


#Delete the first 5 rows
census = census.drop(census.index[[0,1,2,3,4]])
census = census.reset_index()
del census['index']

census = census.rename(columns = {'name':'Zip Code'})
len(census)


## Keep only the variables of interest
census = census[["geoid","Zip Code", "B03002001", "B03002002", "B03002003", "B03002004","B03002012", "B03002013","B03002014"]]

#Rename some columns

census = census.rename(columns = {'B03002001':'Total Population'})
census = census.rename(columns = {'B03002002':'Not Hispanic'})
census = census.rename(columns = {'B03002003':'NonHisp White'})
census = census.rename(columns = {'B03002004':'NonHisp Black'})
census = census.rename(columns = {'B03002012':'Hispanic'})
census = census.rename(columns = {'B03002013':'Hisp White'})
census = census.rename(columns = {'B03002014':'Hisp Black'})

census.head()

,geoid,Zip Code,Total Population,Not Hispanic,NonHisp White,NonHisp Black,Hispanic,Hisp White,Hisp Black
0,86000US90001,90001,57942,5719,435,4936,52223,20376,217
1,86000US90002,90002,51826,12299,407,11002,39527,19166,186
2,86000US90003,90003,70208,16578,433,15606,53630,17681,600
3,86000US90004,90004,63095,31130,11187,2296,31965,14424,109
4,86000US90005,90005,39338,19139,3040,2027,20199,4431,186


In [21]:
#census.dtypes

In [22]:
#crimerate2017.dtypes

In [23]:
#Convert Zip in Census to int
census["Zip Code2"] = census["Zip Code"].astype(str).astype(int)
census.dtypes

geoid               object
Zip Code            object
Total Population     int64
Not Hispanic         int64
NonHisp White        int64
NonHisp Black        int64
Hispanic             int64
Hisp White           int64
Hisp Black           int64
Zip Code2            int64
dtype: object

In [24]:
del census['Zip Code']
census.dtypes

geoid               object
Total Population     int64
Not Hispanic         int64
NonHisp White        int64
NonHisp Black        int64
Hispanic             int64
Hisp White           int64
Hisp Black           int64
Zip Code2            int64
dtype: object

In [25]:
census = census.rename(columns = {'Zip Code2':'Zip Code'})

In [26]:
#MERGE
crime_census = pd.merge(crimerate2017,census,how='inner', on="Zip Code")

In [27]:
print(len(crime_census))
crime_census.to_csv("Data/crime_census.csv")
crime_census.head()

135


,Zip Code,2017 Crime count,geoid,Total Population,Not Hispanic,NonHisp White,NonHisp Black,Hispanic,Hisp White,Hisp Black
0,90001,51,86000US90001,57942,5719,435,4936,52223,20376,217
1,90002,524,86000US90002,51826,12299,407,11002,39527,19166,186
2,90003,1456,86000US90003,70208,16578,433,15606,53630,17681,600
3,90004,267,86000US90004,63095,31130,11187,2296,31965,14424,109
4,90005,260,86000US90005,39338,19139,3040,2027,20199,4431,186


In [28]:
#Calculate the rates
crime_census["Percent NonHisp White"]= (crime_census["NonHisp White"]/crime_census["Total Population"])*100
crime_census["Percent NonHisp Black"]= (crime_census["NonHisp Black"]/crime_census["Total Population"])*100
crime_census["Violent Crime Rate"] = (crime_census["2017 Crime count"]/crime_census["Total Population"])*100000

print(f"The number of observations in this dataframe is: {len(crime_census)}")
crime_census1 = crime_census[crime_census[("Total Population")]!=0]
crime_census1[("Percent NonHisp Black")].isna()
crime_census1
crime_census1.to_csv("Data/crime_census_rates.csv")
print(f"The number of observations in this dataframe (after the removal) is: {len(crime_census1)}")
crime_census1.head()

The number of observations in this dataframe is: 135
The number of observations in this dataframe (after the removal) is: 132


,Zip Code,2017 Crime count,geoid,Total Population,Not Hispanic,NonHisp White,NonHisp Black,Hispanic,Hisp White,Hisp Black,Percent NonHisp White,Percent NonHisp Black,Violent Crime Rate
0,90001,51,86000US90001,57942,5719,435,4936,52223,20376,217,0.750751,8.518864,88.019054
1,90002,524,86000US90002,51826,12299,407,11002,39527,19166,186,0.785320,21.228727,1011.075522
2,90003,1456,86000US90003,70208,16578,433,15606,53630,17681,600,0.616739,22.228236,2073.837739
3,90004,267,86000US90004,63095,31130,11187,2296,31965,14424,109,17.730407,3.638957,423.171408
4,90005,260,86000US90005,39338,19139,3040,2027,20199,4431,186,7.727897,5.152778,660.938533


In [29]:
crime_census1["Percent NonHisp White"].describe()

count    132.000000
mean      35.235826
std       26.648340
min        0.594681
25%        7.897190
50%       32.575811
75%       60.336687
max       84.916045
Name: Percent NonHisp White, dtype: float64

In [30]:
#grouping the counts by area name
#crimedata["area_name"].value_counts()

Maps using Folium

In [31]:
## LAST TRY with the Choropleth
#1. Import the geojson file

#la_shape = os.path.join('Data','zip-code-tabulation-areas-2012.json')
#geo_json_data = json.load(open(la_shape))

MAX_RECORDS = 33139
#Create an empty map for the city of LA

map = folium.Map(location=LA_COORDINATES, zoom_start=12)

#Add a clustered marker
marker_cluster = folium.plugins.MarkerCluster().add_to(map)

for row in crimedata[0:MAX_RECORDS].iterrows():
    popup = row[1]["crm_cd_desc"]
    folium.Marker(location = [row[1]["Latitude"],row[1]["Longitude"]],popup = popup, icon=folium.Icon(color='red', icon='info-sign')).add_to(marker_cluster)

#Lay the shapefile on this empty map
#folium.GeoJson(geo_json_data).add_to(map)
#map.choropleth(
    #geo_data=geo_json_data,
    #data=demographics,
    #columns=['Zip Code', 'Percent NonHisp Black'],
    #key_on='features.properties.external_id',
    #fill_color='YlGn',)
    
folium.LayerControl().add_to(map)
map.save('Results/trial2.html')

In [32]:
#Create the Choropleth maps based on this reference: 
#MAX_RECORDS = 500

#zip_geo = r"acs2016_5yr_B17001_16000US0648816.geojson"

# create empty map zoomed in on Los Angeles
#map = folium.Map(location=LA_COORDINATES, zoom_start=12)

#Add a clustered marker
#marker_cluster = folium.plugins.MarkerCluster().add_to(map)

# Define your threshold scale
#threshold_scale = split_six(crime_census['Percent NonHisp Black'])

#for row in crimedata[0:MAX_RECORDS].iterrows():
    #popup = row[1]["crm_cd_desc"]
    #folium.Marker(location = [row[1]["Latitude"],row[1]["Longitude"]],popup = popup, icon=folium.Icon(color='red', icon='info-sign')).add_to(marker_cluster)

#folium.GeoJson(zip_geo, name = 'crimeagg.json').add_to(map)
#folium.LayerControl().add_to(map)
#map.save('Results/index6.html')

#map.choropleth(
    #geo_data=zip_geo,data=crime_census,
    #columns=['Zip Code', 'Percent NonHisp Black'],
    #key_on='feature.properties.external_id',
    #fill_color='YlGn',
    #threshold_scale=threshold_scale,
    #highlight=False, 
    #reset = True)
#folium.LayerControl().add_to(map)
#map.save('Results/index8.html')

In [33]:
#Now draw a choropleth of percentage of black in the LA by zipcodes
#First, load the data that has the information about the percent of blacks

#demographics = pd.read_csv("Data/crime_census_rates.csv")
#len(demographics)

In [34]:
#demographics.dtypes

In [35]:
#Get the varible max and min
#demographics[("Percent NonHisp Black")].describe()

In [36]:
#colormap = linear.BuGn_03.scale(0.254777,77.302402)
#print(colormap(5.0))
#colormap

In [37]:
#convert the table into a dictionnary, in order to map a feature to it's percent black values.
#percent_dict = demographics.set_index('Zip Code')['Percent NonHisp Black']
#Check if the index is working
#percent_dict[90001]

In [38]:
#Check on the geojson file where the zipcode information is:
#geo_json_data['features'][10]['properties']['external_id']

In [39]:
#folium.GeoJson(
    #geo_json_data,
    #name='demographics',
    #style_function=lambda feature: {
        #'fillColor': colormap(percent_dict[feature['properties']['external_id']]),
        #'color': 'white',
        #'weight': 1,
        #'dashArray': '5, 5',
        #'fillOpacity': 0.9,
   # }
#).add_to(map)

#folium.LayerControl().add_to(map)